<h1> Contact List with Lambda and DynamoDB </h1>

Creates a contact list using AWS Lambda and stores into DB on DynamoDB from prompt inputs on Contact Flows via Amazon Connect.

Followed/copied from Alonso Suarez: <br>
https://gist.github.com/alonch/2887d941689f8dbd560614c8667e6300



In [ ]:
import json
import boto 3

def lambda_handler(event, context):
    return boto3.client('lambda').add_permission(
        Action="lambda:InvokeFunction", 
        FunctionName = "<FUNCTION-THAT-WANT-TO-RUN-FROM-AMAZON-CONNECT>", 
        Principal= 'connect.amazonaws.com',
        SourceAccount='<ACCOUNT-ID>', #check Account Id at https://console.aws.amazon.com/billing/home?#/account
        StatementId = "1"
    )

In [ ]:


dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('contact-list')

def lambda_handler(event, context):
    address = event\
    .get('Details',{})\
    .get('CustomerEndpoint', {})\
    .get('Address', None)
    
    if not address:
        raise('JSON object is invalid')
        
    transferTo = event\.get('Details', {})\.get('Parameters', {})\.get('transferTo', None)
    
    if transferTo: #will update if transferTo is in parameters
        put(address, transferTo)
    else:  #else it will query
        return get(address)
        
    def put(address, transferTo):
        table.put_item(Item={'origin': address, 'transferTo': transferTo})
    
    def get(address):
        response = table.get_item(Key={'origin':address})
        if "Item" not in response:
            raise Exception('not found')

        return response.get('Item')
        
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }